# Mapping of Geolocations onto a 2D space

Reference: https://www.kaggle.com/learn/geospatial-analysis

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import folium
from folium.plugins import HeatMap
from folium import Marker
from folium.features import GeoJson

In [6]:
import pyproj

AttributeError: WKT2_2019

In [3]:
import geopandas as gpd

AttributeError: WKT2_2019

Importing a dataset from GeoLife

In [ ]:
# GEOLIFE_DATA_PATH =r"C:\Users\Tay\Documents\GitHub\Trajectory-Data-Mining\Geolife Trajectories 1.3\Data"
GEOLIFE_DATA_PATH =r"C:\Users\tay.yq.XTRAMAN\Documents\GitHub\Trajectory-Data-Mining\Geolife Trajectories 1.3\Data"

In [ ]:
users = os.listdir(GEOLIFE_DATA_PATH)
print("Number of users in Geolife dataset: " + str(len(users)))

In [ ]:
def readUserTraj(path_to_user):
    '''
    :param path_to_user: a path to the user's dataset according to the Geolife dataset file system
    :return: a dataframe containing all of the user's trajectories
    '''
    trajs = os.listdir(path_to_user + r"\Trajectory")
    col_names = ["lat", "lon", "alt", "date", "time"]
    userTraj = pd.DataFrame(columns=["lat", "lon", "alt", "time"])
    
    for i in range(len(trajs)):
        TRAJ_PATH = path_to_user + r"\Trajectory" + r"\\" + trajs[i]
        traj = pd.read_csv(TRAJ_PATH, skiprows=6, header=None, usecols=[0,1,3,5,6], names=col_names)
        
        traj["full_date"] = traj["date"].str.cat(traj["time"], sep=" ")
        traj["full_date"] = traj["full_date"].apply(lambda x: pd.to_datetime(x))

        traj = traj.drop(columns = ["date", "time"])
        traj = traj.rename(columns = {"full_date": "time"})
        
        userTraj = userTraj.append(traj)
        userTraj = userTraj.reset_index(drop=True)
    
    return userTraj

In [ ]:
%%time
trajDataFrames = [readUserTraj(GEOLIFE_DATA_PATH + r"\\" + users[i]) for i in range(1)]

In [ ]:
print(trajDataFrames[0].shape)
print("-"*50)
trajDataFrames[0].head()

In [ ]:
type(trajDataFrames[0])

Importing a geolocation dataset from Kaggle 

In [ ]:
# Read in the data
full_data = gpd.read_file("Kaggle_Data/geospatial-learn-course-data/DEC_lands/DEC_lands/DEC_lands.shp")

# View the first five rows of the data
full_data.head()

In [ ]:
type(full_data)

GeoDataFrame inherits the same functions as pandas DataFrame. For example, we can select a subset of features, and make a value count.

In [ ]:
data = full_data.loc[:, ["CLASS", "COUNTY", "geometry"]].copy()
data.head()

In [ ]:
data.CLASS.value_counts()

In [ ]:
# Select lands that fall under the "WILD FOREST" or "WILDERNESS" category
wild_lands = data.loc[data.CLASS.isin(['WILD FOREST', 'WILDERNESS'])].copy()
wild_lands.head()

## Plotting the data

In [ ]:
wild_lands.plot(figsize=(15,5))
plt.show()

## Geometry

Every GeoDataFrame contains a special "geometry" column. It contains all of the geometric objects that are displayed when we call the plot() method.

In [ ]:
wild_lands.geometry.head()

While this column can contain a variety of different datatypes, each entry will typically be a Point, LineString, or Polygon.

<img src="images/geopandas_geometry.PNG">

In [ ]:
# Campsites in New York state (Point)
POI_data = gpd.read_file("Kaggle_Data/geospatial-learn-course-data/DEC_pointsinterest/DEC_pointsinterest/Decptsofinterest.shp")
campsites = POI_data.loc[POI_data.ASSET=='PRIMITIVE CAMPSITE'].copy()

# Foot trails in New York state (LineString)
roads_trails = gpd.read_file("Kaggle_Data/geospatial-learn-course-data/DEC_roadstrails/DEC_roadstrails/Decroadstrails.shp")
trails = roads_trails.loc[roads_trails.ASSET=='FOOT TRAIL'].copy()

# County boundaries in New York state (Polygon)
counties = gpd.read_file("Kaggle_Data/geospatial-learn-course-data/NY_county_boundaries/NY_county_boundaries/NY_county_boundaries.shp")

In [ ]:
# Define a base map with county boundaries
ax = counties.plot(figsize=(10,10), color='none', edgecolor='gainsboro', zorder=3)

# Add wild lands, campsites, and foot trails to the base map
wild_lands.plot(color='lightgreen', ax=ax)
campsites.plot(color='maroon', markersize=2, ax=ax)
trails.plot(color='black', markersize=1, ax=ax)

plt.show()

## Coordinate Reference System (CRS)

When we create a GeoDataFrame from a shapefile, the CRS is already imported for us.

Coordinate reference systems are referenced by European Petroleum Survey Group (EPSG) codes.

The following GeoDataFrame uses **EPSG 32630**, which is more commonly called the **"Mercator" projection**. This projection preserves angles (making it useful for sea navigation) and slightly distorts area.

However, when creating a GeoDataFrame from a CSV file, we have to set the CRS. **EPSG 4326 corresponds to coordinates in latitude and longitude.**

In [ ]:
# Load a GeoDataFrame containing regions in Ghana
regions = gpd.read_file("Kaggle_Data/geospatial-learn-course-data/ghana/ghana/Regions/Map_of_Regions_in_Ghana.shp")
print(regions.crs)

### Setting the CRS of a dataframe from a csv file with longitude and latitude.

In [ ]:
# Create a DataFrame with health facilities in Ghana
facilities_df = pd.read_csv("Kaggle_Data/geospatial-learn-course-data/ghana/ghana/health_facilities.csv")

# Convert the DataFrame to a GeoDataFrame
facilities = gpd.GeoDataFrame(facilities_df, geometry=gpd.points_from_xy(facilities_df.Longitude, facilities_df.Latitude))

# Set the coordinate reference system (CRS) to EPSG 4326
facilities.crs = {'init': 'epsg:4326'}

# View the first five rows of the GeoDataFrame
facilities.head()

### Changing the CRS of a dataframe from a csv file with longitude and latitude.

In [ ]:
# Create a map
ax = regions.plot(figsize=(8,8), color='whitesmoke', linestyle=':', edgecolor='black')
facilities.to_crs(epsg=32630).plot(markersize=1, ax=ax)

plt.show()

The `to_crs()` method modifies only the "geometry" column: all other columns are left as-is.



In [ ]:
# The "Latitude" and "Longitude" columns are unchanged
facilities.to_crs(epsg=32630).head()

## Proximity Analysis

We can do the following using Geopandas

- measure the distance between points on a map, and
- select all points within some radius of a feature.

Dataset from the US Environmental Protection Agency (EPA) that tracks releases of toxic chemicals in Philadelphia, Pennsylvania, USA.

In [ ]:
releases = gpd.read_file("Kaggle_Data/geospatial-learn-course-data/toxic_release_pennsylvania/toxic_release_pennsylvania/toxic_release_pennsylvania.shp") 
releases.head()

Dataset that contains readings from air quality monitoring stations in the same city.

In [ ]:
stations = gpd.read_file("Kaggle_Data/geospatial-learn-course-data/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations.shp")
stations.head()

## Measuring distance¶
To measure distances between points from two different GeoDataFrames, we first have to make sure that they use the same coordinate reference system (CRS). Thankfully, this is the case here, where both use EPSG 2272.

In [ ]:
print(stations.crs)
print(releases.crs)

We also check the CRS to see which units it uses (meters, feet, or something else). In this case, EPSG 2272 has units of feet. (If you like, you can check this [here](https://epsg.io/2272).)

It's relatively straightforward to compute distances in GeoPandas. The code cell below **calculates the distance (in feet) between a relatively recent release incident in `recent_release` and every `station` in the stations GeoDataFrame.**

In [ ]:
# Select one release incident in particular
recent_release = releases.iloc[360]

# Measure distance from release to each station
distances = stations.geometry.distance(recent_release.geometry)
distances

Using the calculated distances, we can obtain statistics like the mean distance to each station.

In [ ]:
print('Mean distance to monitoring stations: {} feet'.format(distances.mean()))

Or, we can get the closest monitoring station.

In [ ]:
print('Closest monitoring station ({} feet):'.format(distances.min()))
print(stations.iloc[distances.idxmin()][["ADDRESS", "LATITUDE", "LONGITUDE"]])

## Creating a buffer
If we want to understand all points on a map that are some radius away from a point, the simplest way is to create a buffer.

The code cell below creates a GeoSeries `two_mile_buffer` containing 12 different Polygon objects. Each polygon is a buffer of 2 miles (or, 2*5280 feet) around a different air monitoring station.

In [ ]:
two_mile_buffer = stations.geometry.buffer(2*5280)
two_mile_buffer.head()

In [ ]:
type(two_mile_buffer)

In [59]:
two_mile_buffer.to_crs(epsg=4326)

0     POLYGON ((-75.05994 40.00773, -75.06024 40.004...
1     POLYGON ((-75.19926 40.04963, -75.19955 40.046...
2     POLYGON ((-74.97541 40.07117, -74.97571 40.068...
3     POLYGON ((-75.12780 39.94366, -75.12809 39.940...
4     POLYGON ((-75.04271 39.99081, -75.04300 39.987...
5     POLYGON ((-75.14952 39.92195, -75.14981 39.919...
6     POLYGON ((-75.10576 39.95992, -75.10605 39.957...
7     POLYGON ((-75.18318 39.88418, -75.18347 39.881...
8     POLYGON ((-74.94598 40.05306, -74.94628 40.050...
9     POLYGON ((-75.14782 39.91194, -75.14811 39.909...
10    POLYGON ((-75.16959 39.98813, -75.16988 39.985...
11    POLYGON ((-75.11226 39.95190, -75.11255 39.949...
dtype: geometry

We use `folium.GeoJson()` to plot each polygon on a map. Note that since folium requires coordinates in latitude and longitude, we have to convert the CRS to EPSG 4326 before plotting.

In [68]:
# Create map with release incidents and monitoring stations
m = folium.Map(location=[39.9526,-75.1652], zoom_start=11)
HeatMap(data=releases[['LATITUDE', 'LONGITUDE']], radius=15).add_to(m)
for idx, row in stations.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(m)


# Plot each polygon on the map
GeoJson(two_mile_buffer.to_crs(epsg=4326)).add_to(m)

# Show the map
m

RuntimeError: b'no arguments in initialization list'

In [72]:
temp = two_mile_buffer.to_crs(epsg=4326)

In [78]:
# Plot each polygon on the map
temp = GeoJson(temp)

RuntimeError: b'no arguments in initialization list'

In [71]:
temp.add_to(m)

AttributeError: 'GeoSeries' object has no attribute 'add_to'

In [46]:
m